In [1]:
import pandas as pd
import json
from collections import defaultdict, Counter
import glob
from analysis_func.text_preproc import preproc_text
import numpy as np
from collections import defaultdict, Counter
import glob
import json
from random import sample
import sklearn

import re
import string
import warnings
from bs4 import BeautifulSoup
import pickle as pkl

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, balanced_accuracy_score
import matplotlib.pyplot as plt

from pandarallel import pandarallel
pandarallel.initialize()

INFO: Pandarallel will run on 128 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
with open("../lite_data/initial_bert_embeddings.pkl", "rb") as rfile:
    berted_labeled = pkl.load(rfile)

In [3]:
labeled_data = pd.read_csv("../lite_data/first_complete_annotated_data.csv", header=0)

In [4]:
labeled_data['bin_label'] = labeled_data['label'].apply(lambda x: 1 if x=='recovery' else 0)

In [5]:
Lab_id2sent = labeled_data.set_index("id").to_dict()['bin_label']

In [6]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('stsb-bert-large')

In [7]:
print("cleaning data")
#text preproessing - filter engligh, hindi, marathi stop words, remove puncts, hash, mentions, urls, weird spaces etc.
labeled_data['clean_text'] = labeled_data['title'].parallel_apply(lambda x: preproc_text(x))

cleaning data


In [8]:
Lab_corpus = labeled_data['clean_text'].tolist()
Lab_corpus_embeddings = model.encode(Lab_corpus)

In [9]:
Lab_labels = np.array(labeled_data['bin_label'].tolist())